# 🇹🇭 PaiNaiDee Backend - Google Colab Deployment

**Deploy the PaiNaiDee Tourism API Backend on Google Colab with ngrok tunnel**

This notebook allows you to run the PaiNaiDee Backend API temporarily on Google Colab with external access via ngrok.

## 📋 What this does:
- ✅ Installs all required dependencies
- ✅ Clones the PaiNaiDee Backend repository
- ✅ Sets up a temporary SQLite database with sample data
- ✅ Starts the Flask API server
- ✅ Creates a public ngrok tunnel for external access
- ✅ Provides you with a public URL to access the API

## ⚠️ Important Notes:
- This is a **temporary deployment** - it will stop when you close this notebook
- Uses **SQLite database** with sample data (not PostgreSQL)
- **ngrok URL changes** each time you run this notebook
- For production use, deploy on Hugging Face Spaces or your own server

## 🚀 How to use:
1. Run all cells below in order (Runtime → Run all)
2. Wait for the setup to complete
3. Copy the ngrok URL from the output
4. Use the URL to access the API (e.g., `https://xxxxx.ngrok.io/api/attractions`)

---

## 🔧 Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install flask flask-cors flask-jwt-extended flask-marshmallow flask-sqlalchemy
!pip install marshmallow marshmallow-sqlalchemy python-dotenv sqlalchemy
!pip install pyngrok

print("✅ Dependencies installed successfully!")

## 📦 Step 2: Clone Repository

In [ ]:
import os
import sys

# Remove existing directory if it exists
if os.path.exists('PaiNaiDee_Backend'):
    !rm -rf PaiNaiDee_Backend

# Clone the repository
!git clone https://github.com/athipan1/PaiNaiDee_Backend.git

# Change to project directory
os.chdir('PaiNaiDee_Backend')

# Add project to Python path
if '/content/PaiNaiDee_Backend' not in sys.path:
    sys.path.append('/content/PaiNaiDee_Backend')

print("✅ Repository cloned and configured!")
print(f"📁 Current directory: {os.getcwd()}")

## 🗄️ Step 3: Setup Database

In [ ]:
import sqlite3
import tempfile
from datetime import datetime

# Create SQLite database with sample data
def setup_colab_database():
    db_path = '/content/painaidee_colab.db'
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create attractions table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS attractions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT,
            province TEXT,
            category TEXT,
            address TEXT,
            latitude REAL,
            longitude REAL,
            rating REAL DEFAULT 0.0,
            image_url TEXT,
            contact_phone TEXT,
            contact_email TEXT,
            website TEXT,
            opening_hours TEXT,
            entrance_fee TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Insert comprehensive sample data
    sample_attractions = [
        ('วัดพระแก้ว', 'วัดพระแก้วมรกต วัดที่สำคัญที่สุดในประเทศไทย ตั้งอยู่ในพระบรมมหาราชวัง', 
         'กรุงเทพมหานคร', 'วัด', 'พระนคร กรุงเทพมหานคร', 13.7563, 100.4925, 4.8, 
         'https://example.com/wat-phra-kaew.jpg', '02-623-5500', 'info@palaces.thai.net', 
         'https://www.palaces.thai.net', '08:30-15:30', '500 บาท'),
        
        ('อุทยานแห่งชาติเขาใหญ่', 'อุทยานแห่งชาติที่เก่าแก่ที่สุดของไทย อุดมด้วยป่าไผ่และสัตว์ป่า', 
         'นครราชสีมา', 'ธรรมชาติ', 'ปากช่อง นครราชสีมา', 14.4333, 101.3667, 4.5,
         'https://example.com/khao-yai.jpg', '044-429-348', 'khaoyai@dnp.go.th', 
         'https://park.dnp.go.th', '06:00-18:00', '40 บาท'),
        
        ('เกาะสมุย', 'เกาะสวยงามในอ่าวไทย มีชายหาดทรายขาวและน้ำใสใส', 
         'สุราษฎร์ธานี', 'ชายหาด', 'เกาะสมุย สุราษฎร์ธานี', 9.5018, 99.9648, 4.6,
         'https://example.com/koh-samui.jpg', '077-421-281', 'tourism@samui.go.th', 
         'https://www.samui.go.th', '24 ชั่วโมง', 'ฟรี'),
        
        ('พระราชวังเก่า', 'พระราชวังเก่าแห่งกรุงเทพมหานคร สถาปัตยกรรมแบบไทยโบราณ', 
         'กรุงเทพมหานคร', 'ประวัติศาสตร์', 'พระนคร กรุงเทพมหานคร', 13.7500, 100.4930, 4.7,
         'https://example.com/grand-palace.jpg', '02-623-5500', 'palace@thai.go.th', 
         'https://www.royalgrandpalace.th', '08:30-15:30', '500 บาท'),
        
        ('วัดพระธาตุดอยสุเทพ', 'วัดบนดอยสุเทพ เชียงใหม่ มีพระธาตุศักดิ์สิทธิ์และวิวสวยงาม', 
         'เชียงใหม่', 'วัด', 'เมือง เชียงใหม่', 18.8046, 98.9217, 4.9,
         'https://example.com/doi-suthep.jpg', '053-295-002', 'doisuthep@wat.th', 
         'https://www.doisuthep.com', '06:00-18:00', '30 บาท'),
        
        ('ตลาดน้ำดำเนินสะดวก', 'ตลาดน้ำดั้งเดิมที่ยังคงเสน่ห์ของวิถีชีวิตไทยโบราณ', 
         'ราชบุรี', 'วัฒนธรรม', 'ดำเนินสะดวก ราชบุรี', 13.5167, 99.9544, 4.3,
         'https://example.com/floating-market.jpg', '032-254-179', 'market@damnoen.com', 
         'https://www.damnoensaduak.com', '07:00-17:00', '20 บาท'),
        
        ('ปราสาทหินพิมาย', 'ปราสาทหินขอมโบราณ อายุกว่า 1,000 ปี สถาปัตยกรรมงดงาม', 
         'นครราชสีมา', 'ประวัติศาสตร์', 'พิมาย นครราชสีมา', 15.2167, 102.4928, 4.4,
         'https://example.com/phimai.jpg', '044-471-568', 'phimai@culture.go.th', 
         'https://www.phimai.com', '08:00-17:00', '100 บาท'),
        
        ('เขาสก', 'ภูเขาสูงในจังหวัดเพชรบุรี มีถ้ำและลิงป่าจำนวนมาก', 
         'เพชรบุรี', 'ธรรมชาติ', 'เมือง เพชรบุรี', 12.9497, 99.9592, 4.2,
         'https://example.com/khao-sok.jpg', '032-428-539', 'khaosok@park.go.th', 
         'https://www.khaosok.com', '08:00-17:00', '200 บาท')
    ]
    
    cursor.executemany('''
        INSERT INTO attractions 
        (name, description, province, category, address, latitude, longitude, rating, 
         image_url, contact_phone, contact_email, website, opening_hours, entrance_fee) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', sample_attractions)
    
    # Create users table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            email TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create reviews table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS reviews (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            place_id INTEGER,
            rating INTEGER CHECK(rating >= 1 AND rating <= 5),
            comment TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (place_id) REFERENCES attractions(id)
        )
    ''')
    
    conn.commit()
    conn.close()
    
    return db_path

# Setup database
db_path = setup_colab_database()
print(f"✅ Database created successfully at: {db_path}")

# Verify data
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM attractions")
count = cursor.fetchone()[0]
conn.close()

print(f"📊 Database contains {count} sample attractions")

## 🌐 Step 4: Setup ngrok

In [ ]:
from pyngrok import ngrok
import os

# Set up environment variables
os.environ['FLASK_ENV'] = 'development'
os.environ['SECRET_KEY'] = 'colab-demo-secret-key-change-in-production'
os.environ['DATABASE_URL'] = f'sqlite:///{db_path}'

# Kill any existing ngrok tunnels
ngrok.kill()

print("✅ ngrok configured successfully!")
print("🔧 Environment variables set for Colab deployment")

## 🚀 Step 5: Start the Application

In [ ]:
import threading
import time
from flask import Flask, jsonify
import sys
import subprocess

# Create a simple Flask app for Colab
def create_colab_app():
    try:
        # Try to import the main app
        from src.app import create_app
        app = create_app('development')
        
        # Add Colab-specific route
        @app.route('/')
        def colab_home():
            return {
                "message": "🇹🇭 PaiNaiDee Backend API running on Google Colab!",
                "description": "Thai Tourism API - Where to go in Thailand",
                "platform": "Google Colab + ngrok",
                "api_base": "/api",
                "sample_endpoints": {
                    "all_attractions": "/api/attractions",
                    "search_attractions": "/api/search?q=วัด",
                    "attraction_by_id": "/api/attractions/1",
                    "health_check": "/health"
                },
                "database": "SQLite with sample data",
                "github": "https://github.com/athipan1/PaiNaiDee_Backend",
                "note": "This is a temporary deployment. The URL will change each time you restart."
            }
        
        @app.route('/health')
        def health():
            return {"status": "healthy", "platform": "google-colab", "timestamp": time.time()}
        
        return app
    
    except Exception as e:
        print(f"❌ Error creating main app: {e}")
        print("🔄 Creating fallback app...")
        
        # Create a simple fallback app
        app = Flask(__name__)
        
        @app.route('/')
        def fallback_home():
            return jsonify({
                "error": "Main app initialization failed",
                "message": "PaiNaiDee Backend - Fallback Mode",
                "platform": "Google Colab (Fallback)",
                "details": str(e),
                "available_endpoints": {
                    "health": "/health",
                    "info": "/info"
                }
            })
        
        @app.route('/health')
        def fallback_health():
            return jsonify({"status": "running", "mode": "fallback"})
        
        @app.route('/info')
        def fallback_info():
            return jsonify({
                "app": "PaiNaiDee Backend",
                "mode": "Fallback",
                "github": "https://github.com/athipan1/PaiNaiDee_Backend",
                "note": "Main app failed to load. Check the error message above."
            })
        
        return app

# Create the Flask app
app = create_colab_app()

print("✅ Flask app created successfully!")

## 🌟 Step 6: Start Server and Create Public URL

In [ ]:
import threading
from pyngrok import ngrok
import time

# Function to run Flask app
def run_flask():
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

# Start Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait a moment for Flask to start
time.sleep(5)

# Create ngrok tunnel
try:
    public_url = ngrok.connect(5000)
    print("🎉 SUCCESS! Your PaiNaiDee Backend API is now running!")
    print("="*60)
    print(f"🌐 Public URL: {public_url}")
    print(f"🏠 Homepage: {public_url}")
    print(f"📋 API Base: {public_url}/api")
    print(f"❤️  Health Check: {public_url}/health")
    print("="*60)
    print("\n📚 Sample API Endpoints:")
    print(f"   • Get all attractions: {public_url}/api/attractions")
    print(f"   • Search attractions: {public_url}/api/search?q=วัด")
    print(f"   • Get specific attraction: {public_url}/api/attractions/1")
    print("\n⚠️  Important Notes:")
    print("   • This URL is temporary and will change when you restart")
    print("   • Keep this notebook running to maintain the server")
    print("   • Uses SQLite database with sample Thai tourism data")
    print("   • For permanent deployment, use Hugging Face Spaces")
    print("\n🔗 GitHub Repository: https://github.com/athipan1/PaiNaiDee_Backend")
    
except Exception as e:
    print(f"❌ Error creating ngrok tunnel: {e}")
    print("💡 Try running this cell again or restart the runtime")

# Keep the server running
print("\n🔄 Server is running... Keep this cell output visible to see the URL")
print("⏹️  To stop the server, restart the runtime or interrupt the kernel")

## 🧪 Step 7: Test the API (Optional)

In [ ]:
import requests
import json

# Get the ngrok URL (you may need to update this with the actual URL from above)
try:
    # Test the API
    base_url = str(public_url)
    
    print("🧪 Testing API endpoints...\n")
    
    # Test homepage
    try:
        response = requests.get(f"{base_url}/")
        print("✅ Homepage test:")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        print("\n" + "="*50 + "\n")
    except Exception as e:
        print(f"❌ Homepage test failed: {e}\n")
    
    # Test health endpoint
    try:
        response = requests.get(f"{base_url}/health")
        print("✅ Health check test:")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        print("\n" + "="*50 + "\n")
    except Exception as e:
        print(f"❌ Health check failed: {e}\n")
    
    # Test attractions endpoint
    try:
        response = requests.get(f"{base_url}/api/attractions")
        if response.status_code == 200:
            data = response.json()
            print("✅ Attractions API test:")
            print(f"Found {len(data.get('attractions', []))} attractions")
            if data.get('attractions'):
                print("First attraction:")
                print(json.dumps(data['attractions'][0], indent=2, ensure_ascii=False))
        else:
            print(f"❌ Attractions API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ Attractions API test failed: {e}")
        
except NameError:
    print("❌ No public URL available. Please run the previous cell first.")

## 📖 Usage Instructions

### 🌐 API Usage
Once your server is running, you can use the public URL to access the API:

1. **Homepage**: `https://your-ngrok-url.ngrok.io/`
2. **All attractions**: `https://your-ngrok-url.ngrok.io/api/attractions`
3. **Search**: `https://your-ngrok-url.ngrok.io/api/search?q=วัด`
4. **Specific attraction**: `https://your-ngrok-url.ngrok.io/api/attractions/1`

### 🔧 Development Tips
- The database contains 8 sample Thai attractions
- All endpoints support CORS for frontend development
- Authentication endpoints are available but may need additional setup
- Check the GitHub repository for complete API documentation

### ⚠️ Limitations
- **Temporary**: Server stops when notebook is closed
- **SQLite**: Uses local SQLite instead of PostgreSQL
- **Performance**: Not optimized for production use
- **URL Changes**: ngrok URL changes each restart

### 🚀 Next Steps
For permanent deployment, consider:
- **Hugging Face Spaces**: Click the deploy button in the GitHub README
- **Heroku/Railway**: Deploy with PostgreSQL database
- **Docker**: Use the provided Docker configuration

---

**🎉 Happy coding with PaiNaiDee Backend! 🇹🇭**